# Sheet 3: Principal Component Analysis

In this exercise sheet we look into how to compute and apply a Principal Component Analysis (PCA).

## Toy 4D Example

We start by loading our toy example. The data is stored as a Numpy array, it is a $2585\times 5$ matrix. The last component of each row is the label, the first four components are the coordinates in 4D. Each label is an integer from  $\{0, 1, 2, 3, 4\}$.

The data contains a noisy 2D plane which is embded into 4D. We would like to represent the data in its _intrinsic_ 2D form.

In [347]:
#!pip install pillow  # install the Python package "pillow"
import numpy as np
import mllab.pca

In [348]:
pca_toy_4d = np.load("data/pca_toy_4d.npy")
y = pca_toy_4d[:, -1]  # labels
x = pca_toy_4d[:, :-1]  # 4D coordinates

print(len(x)) #num of rows
print(len(x[0])) #num of columns

2585
4


Let us plot slices from this 4D data. We provide a helper function for this:

In [349]:
# Show documentation
mllab.pca.plot_toy_slice(x, y, 1)

<IPython.core.display.Javascript object>

We want to remove the noise and recover the 2D information.

In [350]:
"""a = np.array([[9, 2, 3, 4], [1, 2, 3, 4]])
print(a)
#a= np.delete(a, -1, axis=1)
print(a[:,-2::]) # This works I think Hurika
print(size.a)
"""

'a = np.array([[9, 2, 3, 4], [1, 2, 3, 4]])\nprint(a)\n#a= np.delete(a, -1, axis=1)\nprint(a[:,-2::]) # This works I think Hurika\nprint(size.a)\n'

In [351]:
a = np.array([[9, 2, 3],
           [4, 5, 6],
           [7, 0, 5]])
print(a[a[:, 2].argsort()])

[[9 2 3]
 [7 0 5]
 [4 5 6]]


In [352]:
"""X = np.array([[2, 3], [4, 5], [6, 7]])
print(X[:, 0])
print(np.mean(X[:,0]))
print(np.transpose(X))
"""

'X = np.array([[2, 3], [4, 5], [6, 7]])\nprint(X[:, 0])\nprint(np.mean(X[:,0]))\nprint(np.transpose(X))\n'

### Task 3.1

Write an implementation of the function below. Use a singular value decomposition (SVD), but avoid computing it completely since we only need the first $q$ eigenvectors. You can use a NumPy/SciPy function for this.

In [353]:
a = np.array([[9, 2, 3],
           [4, 5, 6],
           [7, 0, 5]])
print(a)
print(a[:, a[0].argsort()])
print(np.delete(a, 0, axis = 0))

[[9 2 3]
 [4 5 6]
 [7 0 5]]
[[2 3 9]
 [5 6 4]
 [0 5 7]]
[[4 5 6]
 [7 0 5]]


In [354]:
from scipy.sparse.linalg import svds

def pca(x, q):
    """
    Compute principal components and the coordinates.
    
    Parameters
    ----------
    
    x: (n, d) NumPy array shouldnt it be (d, n)
    q: int
       The number of principal components to compute.
       Has to be less than `p`.

    Returns
    -------
    
    Vq: (d, q) NumPy array, orthonormal vectors (column-wise)
    xq: (n, q) NumPy array, coordinates for x (row-wise)
    """
    #mean of the dataset
    mu = np.zeros((len(x),))
    for i in range(1,len(mu)):
       mu[i] = np.mean(x[i,:])

    X = x
    #"standardizing" the data 
    for i in range(1,len(mu)):
       X[i,:] = x[i,:] - mu[i] #maybe wrong nxd dim
   
    Z = np.matmul(np.transpose(X),X) 
    
    #returns Eigenvalues (vector) and Eigenvectors (matrix, columnwise)
    answer = np.linalg.eig(Z)
    #print("Dimensions of answer: " + answer.ndim)
    Eigen_vector = answer[0]
    Eigen_matrix = answer[1]
    print("Checking that it is orthonormal: " + str(np.matmul(Eigen_matrix.T,Eigen_matrix)))

    #putting the eigenvalues vector ontop of the matrix
    Eigen_all = np.vstack((Eigen_vector, Eigen_matrix))
    #sorting this matrix wrt the entries of the first row,i.e the eigenvalues
    Sorted_eigen_matrix =Eigen_all[:, Eigen_all[0].argsort()]
    #len(Eigen_all[:,0])-1
    print(Sorted_eigen_matrix)
    
    #removing last column and only looking at the q 
    V = np.delete(Sorted_eigen_matrix, 0, axis = 0)
    #V = np.delete(Sorted_eigen_matrix, -1, axis = 1)
    V = V[0:,-(q)::] #dxq
    print(len(V), len(V[0]))
    #computing lambda_i's
    lam = np.zeros((len(X),q)) # 
    #lam = np.matmul(V.T,X)
    for i in range(len(X)):
       lam[i,:] = np.matmul(V.T, X[i,:])

    return V,lam
       
    
   
    

Now we can compute the 2D dimensional representation of `x` using PCA.

In [355]:
V, xq = pca(x, q=2)
print(xq)

Checking that it is orthonormal: [[ 1.00000000e+00  7.77156117e-16  4.16333634e-16  6.66133815e-16]
 [ 7.77156117e-16  1.00000000e+00 -1.66533454e-16 -5.55111512e-16]
 [ 4.16333634e-16 -1.66533454e-16  1.00000000e+00 -2.77555756e-16]
 [ 6.66133815e-16 -5.55111512e-16 -2.77555756e-16  1.00000000e+00]]
[[ 8.27370302e+01  3.48725377e+05  3.29070042e+06  3.37930126e+06]
 [-4.99590101e-01 -4.78580386e-01  3.64360168e-01  6.23387690e-01]
 [-4.99840464e-01 -5.20909505e-01 -3.78638779e-01 -5.79176548e-01]
 [-5.00293340e-01  5.24185605e-01 -5.94333920e-01  3.48859881e-01]
 [-5.00275739e-01  4.74176712e-01  6.08803362e-01 -3.92732858e-01]]
4 2
[[-34.46954834 -62.72706734]
 [ 23.8849631   71.44980886]
 [ 64.92740915   4.07827538]
 ...
 [-63.46649127  24.69774349]
 [ 31.47955275  29.15340597]
 [ 42.71371834 -43.75049305]]


### Task 3.2 a

And then plot the coordinates `xq`, which are two dimensional. We provide a helper function for this task. Let us check how to use it:

In [360]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()
ax.scatter(xq[:,0], xq[:,1], c='blue')
plt.show()

#mllab.pca.plot_toy_2d(xq.T[:,0], xq.T[:,1])

<IPython.core.display.Javascript object>

Hopefully you appreciate the result. 

### Task 3.2 b

Let us see how PCA handles a non-linear transformation. To test this we map our data into 3D by keeping the y-axis as the new z-axis and bending x-coordinate onto an ellipse.

In [357]:
mllab.pca.map_on_ellipse?

Signature: mllab.pca.map_on_ellipse(xq, a=4, b=1, gap_angle=90)
Docstring:
Map 2D points on bend ellipse in 3D.

Parameters
----------

xq: (n, 2) array-like
    The 2D points
a, b: non-negative, scalar
    Ellipsis axis
gap_angle: scalar in [0,360)
    Degree of the gap the bend ellipse should have.
    E.g. for 0 the data is mapped onto a closed ellipse.

Returns
-------

xyz: (n, 3) array-like
    Mapped 3D points
File:      c:\users\shima\github-classroom\ins-uni-bonn\mllab-exercise-3-benedikt-wilkens-and-shimal-harichurn-master\mllab\pca\__init__.py
Type:      function


In [358]:
xyz = mllab.pca.map_on_ellipse(xq, a=4, b=1, gap_angle=90)

In [359]:
%matplotlib notebook
mllab.pca.plot_toy_3d(xyz, y)

<IPython.core.display.Javascript object>

NotImplementedError: Axes3D currently only supports the aspect argument 'auto'. You passed in 'equal'.

**(Remeber to stop the interactive plot by pressing the shutdown icon!)**

Now apply PCA to our transformed data and plot the result as before.

In [ ]:
%matplotlib inline
# plot code here

Could be worse, but undeniably discomforting. Try different axes lengths and gap sizes of the ellipse. What do you observe?

### Task 3.3 a

We want to see if PCA can improve the accuracy of separating hyperlanes. First compute the singular values of the Iris dataset, then check how many percent of the variance the first two principal components capture.

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_x = iris['data']
iris_y = iris['target']

### Task 3.3 b

Now apply PCA and compute the first two principal components. Plot the projected 2D data in a scatter plot such that the three labels are recognizable. What do you observe?

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_1d_iris(a, b, c):
    """Show a 1D plot of three 1D datasets a, b and c.
    
    Top to bottom plotted in order is a, b, c."""
    left = min(x.min() for x in (a, b, c))
    right = max(x.max() for x in (a, b, c))
    for i, (x, c) in enumerate(((a, 'red'), (b, 'blue'), (c, 'green'))):
        plt.hlines(i * .3, left, right, linestyles='dotted', colors=[(.8,.8,.8,1)])
        plt.eventplot(x, colors=c, linewidths=.5, linelengths=.25, lineoffsets=(2 - i) * .3)
    plt.axis('off')


### Task 3.3 c

Finally, recompute the accurancy as in sheet 1 and compare the results.

In [ ]:
from numpy.linalg import lstsq
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import LinearSVC

def labels(x1, x2):
    return np.concatenate((np.zeros(x1.shape[0], dtype='int'), np.ones(x2.shape[0], dtype='int')))

In [ ]:
# add your code here

## Pedestrian Classification

### Task 3.4 a

__Read the pedestrian dataset into a NumPy array and normalize to [0,1]__

In [ ]:
import mllab.pca
mllab.pca.load_pedestrian_images?

### Task 3.4 b

__Write a function to plot an image__

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

def plot_im(im, ax=None, title=None, max_contrast=False):
    """
    Plot a normalized image.
    
    Parameters
    ----------
    
    im: (1250,) array-like
    """
    # your code here

__Plot 10 randomly chosen images showing a pedestrian, and 10 randomly chosen images not showing a pedestrain.__

In [ ]:
# your code here


### Task 3.5 a

__Compute the PCA of the full training set for $q=200$__

In [ ]:
from sklearn.decomposition import PCA


### Task 3.5 b

__Plot the eigenpedestrian 1-20, 51-60, and 111-120__

### Task 3.6

__Compute the scores for a linear SVM using increasing numbers of principal components__

Use 10 to 200 components in steps of 5. Train the linear SVM with $C=0.01$ and increse the maximum number of iterations for the solver. You can reuse the computed PCA from above.

In [ ]:
from sklearn.svm import LinearSVC


Plot the training and test scores over $q$.

### Task 3.7

## HOG Features

We decided to provide the implementation of the HOG features. (Task 3.7 a). 
Bonus Task (only 3.7 b): Compute the HOG features of the test data. Then apply PCA and calculate the scores

In [ ]:
import numpy as np
import scipy.ndimage as ndimage
from numpy.linalg import norm
from scipy.ndimage.filters import convolve


class HogFeatures:
    def __init__(self, im_shape, n_bins=9, cell_size=8, blk_size=2, unsigned=True, clip_val=.2):
        self.deg_range = np.pi if unsigned else 2 * np.pi
        self.n_bins = n_bins
        self.bins = np.linspace(0, self.deg_range, n_bins, endpoint=False)
        self.bin_size = self.deg_range / n_bins
        self.cell_size = cell_size
        self.blk_size = blk_size
        self.clip_val = clip_val

        self.im_h, self.im_w = im_shape
        x, y = np.arange(self.im_w), np.arange(self.im_h)
        
        # Compute logical cell indices of next lower and upper cell
        # w.r.t. to the cell center
        cells_x = np.arange(-cell_size, self.im_w - (cell_size + 1)/2, cell_size)
        self.n_cells_x = len(cells_x) - (2 if cells_x[-1] >= self.im_w else 1)
        x0 = np.digitize(x, cells_x + cell_size / 2) - 2
        Xc = ((x0 + 1) - .5) * cell_size - .5
        f_x = (x - Xc) / cell_size

        cells_y = np.arange(-cell_size, self.im_h - (cell_size + 1)/2, cell_size)
        self.n_cells_y = len(cells_y) - (2 if cells_y[-1] >= self.im_h else 1)
        y0 = np.digitize(y, cells_y + cell_size / 2) - 2
        Yc = ((y0 + 1) - .5) * cell_size - .5
        f_y = (y - Yc) / cell_size
        
        self.f_x, self.f_y = np.meshgrid(f_x, f_y)
    
    def extract(self, im):
        """
        Extract the HOG features for a image.
        
        Parameters
        ----------
        
        im: ndarray
            An array of shape (height, width, 3).
        """
        im = np.rollaxis(im.reshape(self.im_h, self.im_w, -1), 2)
        dx = convolve(im, [[[1,0,-1]]], mode='constant')
        dy = convolve(im, [[[-1],[0],[1]]], mode='constant')
        grads_mag = norm(np.stack((dx, dy), axis=-1), axis=3)
        max_grads = np.argmax(np.rollaxis(grads_mag, 0, 3), 2)
        Y, X = np.ogrid[:grads_mag.shape[1], :grads_mag.shape[2]]
        grads_dir = np.arctan2(dy[max_grads, Y, X], dx[max_grads, Y, X]) % self.deg_range
        grads_mag = grads_mag[max_grads, Y, X]
        del dx, dy, max_grads, Y, X
        
        # Compute logical bin indices of next lower (<=) and upper bin (>)
        # w.r.t. to the bin center
        bin0 = np.digitize(grads_dir, self.bins + .5 * self.bin_size) - 1
        bin1 = bin0 + 1
        dirc = (bin0 + .5) * self.bin_size
        f_b = (grads_dir - dirc) / self.bin_size
        del grads_dir
        
        bin0 %= self.n_bins
        bin1 %= self.n_bins
        
        f_x, f_y = self.f_x, self.f_y

        hist = np.zeros((self.n_cells_y, self.n_cells_x, self.n_bins))
        bin_labels = np.arange(self.n_bins)
        # Iterate over all cells
        for ci_x in range(self.n_cells_x):
            x_pos = (ci_x * self.cell_size - (self.cell_size + 1) // 2, ci_x * self.cell_size + (self.cell_size + 1) // 2)
            x_pre = slice(max(0, x_pos[0] + self.cell_size), max(0, x_pos[1] + self.cell_size))
            x_pos = slice(max(0, x_pos[0]), x_pos[1])
            for ci_y in range(self.n_cells_y):
                y_pos = (ci_y * self.cell_size - (self.cell_size + 1) // 2, ci_y * self.cell_size + (self.cell_size + 1) // 2)
                y_pre = slice(max(0, y_pos[0] + self.cell_size), max(0, y_pos[1] + self.cell_size))
                y_pos = slice(max(0, y_pos[0]), y_pos[1])
                # Consider all four sourinding cells
                    
                # y-pre x-pre
                m = (y_pre, x_pre)
                g = grads_mag[m] * (1 - f_x[m]) * (1 - f_y[m])
                hist[ci_y, ci_x] += ndimage.sum(g * (1 - f_b[m]), bin0[m], bin_labels) + \
                    ndimage.sum(g * f_b[m], bin1[m], bin_labels)
                # y-pos x-pre
                m = (y_pos, x_pre)
                g = grads_mag[m] * (1 - f_x[m]) * f_y[m]
                hist[ci_y, ci_x] += ndimage.sum(g * (1 - f_b[m]), bin0[m], bin_labels) + \
                    ndimage.sum(g * f_b[m], bin1[m], bin_labels)
                # y-pre x-pos
                m = (y_pre, x_pos)
                g = grads_mag[m] * f_x[m] * (1 - f_y[m])
                hist[ci_y, ci_x] += ndimage.sum(g * (1 - f_b[m]), bin0[m], bin_labels) + \
                    ndimage.sum(g * f_b[m], bin1[m], bin_labels)
                # y-pos x-pos
                m = (y_pos, x_pos)
                g = grads_mag[m] * f_x[m] * f_y[m]
                hist[ci_y, ci_x] += ndimage.sum(g * (1 - f_b[m]), bin0[m], bin_labels) + \
                    ndimage.sum(g * f_b[m], bin1[m], bin_labels)
        
        n_blks_x = self.n_cells_x + 1 - self.blk_size
        n_blks_y = self.n_cells_y + 1 - self.blk_size
        features = np.zeros((n_blks_x, n_blks_y, self.blk_size ** 2 * self.n_bins))
        for bi_x in range(n_blks_x):
            for bi_y in range(n_blks_y):
                blk = hist[bi_y:bi_y+self.blk_size, bi_x:bi_x+self.blk_size].copy()
                blk_norm = norm(blk.flatten())
                if blk_norm > 0:
                    blk /= blk_norm
                np.clip(blk, None, self.clip_val, out=blk)
                blk_norm = norm(blk.flatten())
                if blk_norm > 0:
                    blk /= blk_norm
                features[bi_x, bi_y] = blk.ravel()
        return features.flatten()


To test you Python implementation we provide some intermediate steps. The array `image` is the input, and `steps` contains the values of the inner variables of the HOG algorithm. The image is grayscale.

In [ ]:
image, steps = mllab.pca.hog_test_data()

In [ ]:
# Use Python implementation
hog = HogFeatures((100, 50))
print("Use Python implementation")


### Task 3.7 b 
__Compute the HOG features for the training data, then compute the PCA for $q=200$.__

In [ ]:
#! pip install tqdm
from tqdm import tqdm
hog_train_features = []
for i in tqdm(range(int_train_features.shape[0])):
    im = int_train_features[i].reshape(100, 50, 3)
    hog_train_features.append(hog.extract(im))
print("Computed HoG.")

q = 200
hog_train_features = np.array(hog_train_features)
hog_train_pca = PCA(n_components=q)
hog_train_pca.fit(hog_train_features)

__Compute and plot the scores as above, but this time use the HOG features.__

In [ ]:
# compute hog features of test data
hog_test_features = []

# compute scores


In [ ]:
plt.figure(figsize=(10,10))
plt.title("HOG + PCA + Linear SVM")
# plot the training and testing scores

plt.xlabel('number of used pc'); plt.ylabel('accuracy');plt.legend()